<a href="https://colab.research.google.com/github/sdalal1/Visual-Odometry/blob/main/Visual_Odometry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import progressbar

In [ ]:
file_path = '../dataset/sequences/00/image_0/'
left_images = os.listdir(file_path) # list of strings with names of images
print(len(left_images))

In [ ]:
# the way he has his data setup is different from the default way mine
# is setup, not sure why. I'm going to keep this cell for now and all
# of the rest of the lidar stuff, but note that for now it will not work
file_path = '../dataset/sequences/00/'
velodyne_files = os.listdir(file_path + 'velodyne/')
pointcloud = np.fromfile(filepath + 'velodyne/' + velodyne_files[0],
                         dtype=np.float32)

In [ ]:
pointcloud = pointcloud.reshape(-1, 4) # 4 columns for x, y, z, intensity

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')

xs = pointcloud[:, 0][::20]
ys = pointcloud[:, 1][::20]
zs = pointcloud[:, 2][::20]

ax.set_box_aspect([np.ptp(xs), np.ptp(ys), np.ptp(zs)])
ax.scatter(xs, ys, zs, s=0.01)
ax.grid(False)
ax.axis('off')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

ax.view_init(elev=40, azim=180)

In [ ]:
# I think this cell can be deleted
calib = pd.read_csv('../dataset/sequences/00/calib.txt', delimiter=" ", header=None, index_col=0)
calib
# Tr = np.array(calib.iloc[4]).reshape((3, 4))
# print(Tr.round(4))

In [ ]:
class Dataset_Handler():

    def __init__(self, sequence, lidar=True, progress_bar=True, low_memory=True):

        self.lidar = lidar
        self.low_memory = low_memory

        # I'm commenting out the poses stuff here, I think this has something
        # with the winrar or whatever technique he uses to unzip his data.
        # all of the lidar and ground truth poses and everything are all
        # in one "dataset" folder, so that's why he writes his code this way.
        # we're not going to have that unless we curate that file structure
        # ourselves, and as of video 2 he doesn't use this "poses" variable
        # at all so I'm commenting everyting out
        self.seq_dir = f"../dataset/sequences/{sequence}/"
        # self.poses_dir = f"../dataset/poses/{sequence}.txt"

        self.left_image_files = os.listdir(self.seq_dir + 'image_0')
        self.right_image_files = os.listdir(self.seq_dir + 'image_1')
        self.num_frames = len(self.left_image_files)
        if lidar:
            self.velodyne_files = os.listdir(self.seq_dir + 'velodyne')
            self.lidar_path = self.seq_dir + 'velodyne/'

        # poses = pd.read_csv(self.poses_dir, delimiter=" ", header=None)
        self.gt = np.zeros((self.num_frames, 3, 4))

        # for i in range(len(poses)):
        #     self.gt[i] = np.array(poses.iloc[i]).reshape((3, 4))

        calib = pd.read_csv(self.seq_dir + 'calib.txt', delimiter=" ", header=None, index_col=0)
        self.P0 = np.array(calib.loc['P0:']).reshape((3, 4))
        self.P1 = np.array(calib.loc['P1:']).reshape((3, 4))
        self.P2 = np.array(calib.loc['P2:']).reshape((3, 4))
        self.P3 = np.array(calib.loc['P3:']).reshape((3, 4))
        if lidar:
            self.Tr = np.array(calib.loc['Tr: ']).reshape((3, 4))

        if low_memory:
            self.reset_frames
            self.first_image_left = cv2.imread(self.seq_dir + 'image_0/' + 
                                               self.left_image_files[0], 0)
            self.first_image_right = cv2.imread(self.seq_dir + 'image_0/' + 
                                               self.right_image_files[0], 0)
            self.second_image_left = cv2.imread(self.seq_dir + 'image_0/' + 
                                               self.left_image_files[1], 0)
            if lidar:
                self.first_pointcloud = np.fromfile(self.lidar_path + self.velodyne_files[0],
                                                    dtype=np.float32)
            self.imheight = self.first_image_left.shape[0]
            self.imwidth = self.first_image_left.shape[1]
        else:
            self.images_left = []
            self.images_right = []
            self.pointclouds = []
            if progress_bar:
                bar = progressbar.ProgressBar(maxval=self.num_frames)
                bar.start()
            for i, name_left in enumerate(self.left_image_files):
                name_right = self.right_image_files[i]
                self.images_left.append(cv2.imread(self.seq_dir + 'image_0/' + name_left))
                self.images_right.append(cv2.imread(self.seq_dir + 'image_1/' + name_right))
                if lidar:
                    pointcloud = np.fromfile(self.lidar_path + self.velodyne_files[i],
                                             dytype=np.float32, count=-1).reshape((-1, 4))
                    self.pointclouds.append(pointcloud)
                if progress_bar:
                    bar.update(i+1)
                self.imheight = self.images_left[0].shape[0]
                self.imwidth = self.images_right[0].shape[1]

    def reset_frames(self):
        self.images_left = (cv2.imread(self.seq_dir + 'image_0/' + name_left, 0)
                            for name_left in self.left_image_files)
        self.images_right= (cv2.imread(self.seq_dir + 'image_1/' + name_right, 0)
                            for name_right in self.left_image_files)
        if lidar:
            self.pointcloud = (np.fromfile(self.lidar_path + velodyne_file, dtype=np.float32).reshape((-1, 4))
                               for velodyne_file in self.velodyne_files)
        pass
        

In [ ]:
handler = Dataset_Handler("00", lidar=False, low_memory=False)